In [3]:
%matplotlib widget
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
from shapely.geometry import Point

# The main two projections so we can switch between them
mercator_prj = {'init': 'epsg:3395'}
platte_prj = {'init': 'epsg:4326'}

# Load the California shapefile
CA = geopandas.read_file('CA_State/CA_State_TIGER2016.shp')

# Load the reservoir information and create a Coordinates column
reservoirs = pd.read_csv('../data/reservoirs.csv', index_col='Name')
reservoirs['Coordinates'] = list(zip(reservoirs.Longitude, reservoirs.Latitude))
reservoirs['Coordinates'] = reservoirs['Coordinates'].apply(Point)

# Create the GeoDataFrame for plotting
gdf = geopandas.GeoDataFrame(reservoirs, geometry='Coordinates')
# Set the initial projection
gdf.crs = platte_prj
# Re-project
gdf = gdf.to_crs(mercator_prj)

fig, ax = plt.subplots()

# Plot the California shapefile as an outline
CA.plot(ax=ax, color='white', edgecolor='black')

# Get the X,Y coordinates of the points and plot them as a scatter
# This makes it easier to index them later and check if we're hovering over them
gdf_x = pd.Series((pt.x for pt in gdf['Coordinates'].get_values()))
gdf_y = pd.Series((pt.y for pt in gdf['Coordinates'].get_values()))
sc = ax.scatter(x=gdf_x, y=gdf_y)
ax.set_title('Reservoir Sites')

annot = ax.annotate("", xy=(0,0), xytext=(20,20), textcoords="offset points",
                    bbox=dict(boxstyle="round", fc="w"),
                    arrowprops=dict(arrowstyle="->"))
annot.set_visible(False)
                    
def update_annot(ind):
    index = ind["ind"][0]
    pos = sc.get_offsets()[index]
    annot.xy = pos
    text = "{}\nElevation: {} ft".format(reservoirs.index[index], reservoirs.iloc[index]["Elevation"])
    annot.set_text(text)
    annot.get_bbox_patch().set_facecolor('blue')
    annot.get_bbox_patch().set_alpha(0.4)
    
def hover(event):
    vis = annot.get_visible()
    if event.inaxes == ax:
        cont, ind = sc.contains(event)
        if cont:
            update_annot(ind)
            annot.set_visible(True)
            fig.canvas.draw_idle()
        else:
            if vis:
                annot.set_visible(False)
                fig.canvas.draw_idle()
                
fig.canvas.mpl_connect("motion_notify_event", hover)

# Show plot
plt.show()

FigureCanvasNbAgg()